In [1]:
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.svm import SVC

In [3]:
from sklearn.metrics import accuracy_score, f1_score
def accuracy_f_score(y_pred,y_true):
    print(f"Accuracy score: {round(accuracy_score(y_true, y_pred) * 100,2)}%")
    print('\033[92m' + f"F1 score: {f1_score(y_true, y_pred)}" + '\033[0m')
    

def in_city(x_pred,y_pred):
    targets = []
    
    if (3750901.5068 <= x_pred <= 3770901.5069) and (-19268905.6133 <= y_pred <= -19208905.6133):
        return 1
    else:
        return 0

def sigmoid(x):
    e = np.exp(1)
    y = 1/(1+e**(-x))
    return y


def journey_time(x,y):
    """
    Compute journey time in seconds.
    """
    x = pd.to_datetime(x)
    y = pd.to_datetime(y)
    return (y-x).total_seconds()

def to_binary(x):
    result = []
    for n in x:
        result.append(np.argmax(n))
    return result

In [4]:
df = pd.read_csv('data_train/final_locations.csv')

df["dpc"] = list(map(sigmoid,df["dist_pct_ch"])) #addresses skewness
df.set_index("trajectory_id", inplace=True)
df["final_loc"] = list(map(in_city, df["x_exit"], df["y_exit"]))

X = df.drop(["vmax","vmin","vmean","time_entry","time_exit","hash","dist_pct_ch",
         "x_home","y_home","dist","nj","prev_tr","x_exit","y_exit","final_loc"], axis=1).values
y = df["final_loc"].values
df.head()

,hash,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,dist,net_tr,prev_tr,x_home,y_home,nj,dist_pct_ch,j_time,dpc,final_loc
trajectory_id,,,,,,,,,,,,,,,,,,,,
traj_0000a8602cf2def930488dee7cdad104_1_5,0000a8602cf2def930488dee7cdad104_1,15:02:31,15:18:33,NaN,NaN,NaN,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,45797.982227,99463.898797,3544.948847,3.751014e+06,-1.909398e+07,6.0,-0.071843,962.0,0.482047,0
traj_0000cf177130469eeac79f67b6bcf3df_9_3,0000cf177130469eeac79f67b6bcf3df_9,15:00:32,15:29:48,1.149404,1.149404,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,29603.985176,-1056.813994,270.043451,3.749450e+06,-1.926506e+07,4.0,-0.009039,1756.0,0.497740,0
traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,0001f97b99a80f18f62e2d44e54ef33d_3,14:34:35,15:19:51,30.167742,30.167742,30.167742,3.758738e+06,-1.937594e+07,3.769687e+06,-1.914258e+07,137051.659155,-1867.319643,-1867.319643,3.771461e+06,-1.910413e+07,2.0,0.013813,2716.0,0.503453,0
traj_0002124248b0ca510dea42824723ccac_31_10,0002124248b0ca510dea42824723ccac_31,15:28:54,15:28:54,NaN,NaN,NaN,3.767866e+06,-1.917797e+07,3.767866e+06,-1.917797e+07,61336.955341,5460.552001,-59655.060438,3.765544e+06,-1.917227e+07,9.0,35.468958,0.0,1.000000,0
traj_000219c2a6380c307e8bffd85b5e404b_23_16,000219c2a6380c307e8bffd85b5e404b_23,15:08:05,15:08:05,NaN,NaN,NaN,3.747641e+06,-1.922695e+07,3.747641e+06,-1.922695e+07,17851.785279,-7113.420678,0.000000,3.760336e+06,-1.922818e+07,8.0,0.000000,0.0,0.500000,0


In [ ]:
clf = SVC(kernel='linear', C=.1, gamma=0.001,shrinking=False,probability=True)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=410,shuffle=True)
clf.fit(x_train, y_train)

pred = clf.predict(x_test)

In [ ]:
accuracy_f_score(pred,y_test)